# Rafael Andreolli Prochnow  RA:176059
# Tema quatro: Ração 
## Biblioteca

In [1]:
# ! pip install ortools

In [2]:
from ortools.linear_solver import pywraplp

### Dados do problema 

In [3]:
solver = pywraplp.Solver('Modelo de Ração Animal', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

In [4]:
# Composição nutritiva dos ingredientes
ingredientes = [
    [0.002, 0.01, 0.03],
    [0.5  , 0.09, 0   ],
    [0.008, 0.02, 0.02],
]
# Custo dos ingredientes
obj_custo = [15, 20, 8]

In [5]:
# número de ingredientes
n_ingr = 3
# número de nutrientes
n_nutri = 3

In [6]:
# Porcentagem máxima de cada nutriente 
max_nutri = [0.012, solver.infinity(), 0.2]
# Porcentagem mínima de cada nutriente 
min_nutri = [0.008, 0.22, 0] 
demanda = 1000

In [7]:
# Variáveis
y = {}
for i in range(n_ingr):
    y[i] = solver.NumVar(0,solver.infinity(),'y'+str(i+1))

In [8]:
y

{0: y1, 1: y2, 2: y3}

## Restrições

**Total de ração**:

$y_t = 1000$

---

**Total de ração em função dos ingredientes**:

$y_1 + y_2 + y_3 = y_t$ 

In [9]:
solver.Add(solver.Sum(y.values()) == demanda,'T')
print(solver.Sum(y.values()) == demanda)

(y1 + y2 + y3) == 1000


**Limite de nutrientes da ração**:

Cálcio: $0.008*y_t \leq x_{1,1} + x_{2,1} + x_{3,1} \leq 0.012*y_t$

Proteína: $0.22*y_t \leq x_{1,2} + x_{2,2}$

Carbo: $x_{1,3} + x_{2,3} + x_{3,3} \leq 0.20*y_t$


In [10]:
# Limite mínimo de nutrientes da ração 
for i in range(n_nutri):
    solver.Add(solver.Sum(ingredientes[i][j]*y[j] for j in range(n_ingr)) >= min_nutri[i]*demanda, name='RT1 '+str(i+1))
    print(solver.Sum(ingredientes[i][j]*y[j] for j in range(n_ingr)) >= min_nutri[i]*demanda)

((0.002 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C4F0>) + (0.01 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C100>) + (0.03 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C130>)) >= 8.0
((0.5 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C640>) + (0.09 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C220>) + (0 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C340>)) >= 220.0
((0.008 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C700>) + (0.02 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C4F0>) + (0.02 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F17010C400>)) >= 0


In [11]:
# Limite mínimo de nutrientes da ração 
for i in range(n_nutri):
    solver.Add(solver.Sum(ingredientes[i][j]*y[j] for j in range(n_ingr)) <= max_nutri[i]*demanda, name='RT2 '+str(i+1))
    print(solver.Sum(ingredientes[i][j]*y[j] for j in range(n_ingr)) <= max_nutri[i]*demanda)

((0.002 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F170111580>) + (0.01 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F170111190>) + (0.03 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F1701111C0>)) <= 12.0
Trivial inequality (always true)
((0.008 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F170111670>) + (0.02 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F170111580>) + (0.02 * <ortools.linear_solver.linear_solver_natural_api.VariableExpr object at 0x000001F170111490>)) <= 200.0


**Função objetivo**:

Min $15*y_1 + 20*y_2 + 8*y_3$

In [12]:
obj_custo

[15, 20, 8]

In [13]:
# Função objetivo
obj_terms = [obj_custo[i]*y[i] for i in range(n_ingr)]
solver.Minimize(solver.Sum(obj_terms))

In [14]:
# Resolve
status = solver.Solve()

In [16]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print('Custo total = ', solver.Objective().Value(), '\n')
    nome_ingredientes = ['Custo da Soja = ',
                    'Custo do Milho = ',
                    'Custo da Cana = ']
    for i in range(n_ingr):
        print(nome_ingredientes[i], y[i].solution_value())

Custo total =  12500.000000000002 

Custo da Soja =  642.857142857143
Custo do Milho =  0.0
Custo da Cana =  357.1428571428571
